In [607]:
import pandas as pd
import numpy as np

df = pd.read_csv('train.csv')
df.drop(columns=['ID'], inplace=True)

print(df.head())

    timestamp item corporation location  supply(kg)  price(원/kg)
0  2019-01-01   TG           A        J         0.0          0.0
1  2019-01-02   TG           A        J         0.0          0.0
2  2019-01-03   TG           A        J     60601.0       1728.0
3  2019-01-04   TG           A        J     25000.0       1408.0
4  2019-01-05   TG           A        J     32352.0       1250.0


In [608]:
# 나중에 test set에 넣어주어야 하기 때문에 만들어 둔 리스트

# 월별 과일별 price 평균
avg_m = [[] for _ in range(5)]

# 년별 과일별 price 평균
avg_y = [[] for _ in range(5)]

# 요일별 과일별 price 평균
avg_d = [[] for _ in range(5)]

# location 과일별 price 평균
avg_l = [[] for _ in range(5)]

# corporation 과일별 price 평균
avg_c = [[] for _ in range(5)]

In [609]:
range_list_m = [i for i in range(1, 13)]
range_list_y = [i for i in range(2019, 2024)]
range_list_d = ['Monday', 'Tuesday', 'Wendsday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
range_list_l = ['J', 'S']
range_list_c = ['A', 'B', 'C', 'D', 'E', 'F']

# df, 처리할 컬럼, 컬럼값, 저장할 리스트
def avg_cal (df, col, range_list, avg_list) :
    for item_type in ['TG', 'CR', 'CB', 'RD', 'BC']:
        for i in range_list :
            condition = (df['item'] == item_type) & (df[col] == i)
            avg_col = col + "_avg"
            selected_rows = df[condition]
            if not selected_rows.empty :
                avg_value = selected_rows[avg_col].iloc[0]
            else : 
                avg_value = 0
            if item_type == 'TG':
                avg_list[0].append(avg_value)
            elif item_type == 'CR':
                avg_list[1].append(avg_value)
            elif item_type == 'CB':
                avg_list[2].append(avg_value)
            elif item_type == 'RD':
                avg_list[3].append(avg_value)
            elif item_type == 'BC':
                avg_list[4].append(avg_value)


In [610]:
def preprocessing_for_train(df, whether) :
    
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['year'] = pd.to_datetime(df['timestamp']).dt.year
    df['month'] = pd.to_datetime(df['timestamp']).dt.month
    df['day'] = pd.to_datetime(df['timestamp']).dt.day
    df['day_of_week'] = df['timestamp'].dt.day_name()
    
    if whether[0] == 1 :
        df['month_avg'] = df.groupby(['month', 'item'])['price(원/kg)'].transform('mean')
        avg_cal(df, 'month', range_list_m, avg_m)
    
    if whether[1] == 1 :
        df['year_avg'] = df.groupby(['year', 'item'])['price(원/kg)'].transform('mean')
        avg_cal(df, 'year', range_list_y, avg_y)    
    
    if whether[2] == 1 :
        df['day_of_week_avg'] = df.groupby(['day_of_week', 'item'])['price(원/kg)'].transform('mean')
        avg_cal(df, 'day_of_week', range_list_d, avg_d)  
        
    if whether[3] == 1 :
        df['location_avg'] = df.groupby(['location', 'item'])['price(원/kg)'].transform('mean')   
        avg_cal(df, 'location', range_list_l, avg_l) 
        
    if whether[4] == 1 :
        df['corporation_avg'] = df.groupby(['corporation', 'item'])['price(원/kg)'].transform('mean')      
        avg_cal(df, 'corporation', range_list_c, avg_c)     
        
    if whether[5] == 1 :
        df['sin_timestamp'] = np.sin(2 * np.pi * df['timestamp'].dt.dayofyear / 365)
        df['cos_timestamp'] = np.cos(2 * np.pi * df['timestamp'].dt.dayofyear / 365)
        
    if whether[6] == 0 :
        df.drop(columns=['month'], inplace=True)
        
    if whether[7] == 0 :
        df.drop(columns=['day'], inplace=True)
        
    if whether[8] == 0 :
        df.drop(columns=['year'], inplace=True) 
        
    if whether[9] == 1:
        df = pd.get_dummies(df, columns=['corporation'])
    else:
        df.drop(columns=['corporation'], inplace=True)
    
    if whether[10] == 1 :
        df = pd.get_dummies(df, columns=['location']) 
    else :  
        df.drop(columns=['location'], inplace=True)    
        
    if whether[11] == 1 :
        df = pd.get_dummies(df, columns=['day_of_week']) 
    else :  
        df.drop(columns=['day_of_week'], inplace=True)     
    
    df.drop(columns=['supply(kg)'], inplace=True)  
    # df.drop(columns=['timestamp'], inplace=True) 
    df = pd.get_dummies(df, columns=['item']) 
                
    
    return df

In [611]:
import itertools
binary_combinations = list(itertools.product([0, 1], repeat=12))

In [612]:
from sklearn.metrics import mean_squared_error
import math
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

def modeling(df):
    df['day_of_week'] = df['timestamp'].dt.dayofweek
    march_2019_data_indices = df[(df['timestamp'] >= '2019-03-01') & (df['timestamp'] < '2019-04-01')].index.tolist()
    
    col_list = df.columns
    col_list = [col for col in col_list if col != 'price(원/kg)' and col != 'timestamp']

    X = df[col_list]
    Y = df['price(원/kg)']

    X_train = df.loc[~df.index.isin(march_2019_data_indices), col_list]
    Y_train = df.loc[~df.index.isin(march_2019_data_indices), 'price(원/kg)']

    X_test = df.loc[df.index.isin(march_2019_data_indices), col_list]
    Y_test = df.loc[df.index.isin(march_2019_data_indices), 'price(원/kg)']
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    model_lgbm = LGBMRegressor()
    model_lgbm.fit(X_train_scaled, Y_train)
    
    predictions_lgbm = model_lgbm.predict(X_test_scaled)
    
    predictions_lgbm[X_test['day_of_week'] == 6] = 0
    
    mse_lgbm = mean_squared_error(Y_test, predictions_lgbm)
    
    return mse_lgbm


In [613]:
min_values = [float('inf')] * 5
whether_min_values = [[0] * 12] * 5
count = 0 

for whether in binary_combinations :
    count = count + 1
    df_ = df.copy() 
    total_df = preprocessing_for_train(df_, whether)
    mse = modeling(total_df)
    rmse = math.sqrt(mse)
        
    if rmse < max(min_values):
        min_index = min_values.index(max(min_values))
        min_values[min_index] = rmse
        whether_min_values[min_index] = whether    
        
    print(f'\r{count}, MSE: {mse}', end='', flush=True)
    
print("\nTop 5 Minimum RMSE Values:")
for i, (min_value, whether_min_value) in enumerate(zip(min_values, whether_min_values), 1):
    print(f"{i}. RMSE: {min_value}, Corresponding Whether: {whether_min_value}")

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001727 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 23
[LightGBM] [Info] Number of data points in the train set: 58188, number of used features: 6
[LightGBM] [Info] Start training from score 1126.812143
1, MSE: 1094094.4280625181[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003065 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 44
[LightGBM] [Info] Number of data points in the train set: 58188, number of used features: 13
[LightGBM] [Info] Start training from score 1126.812143
2, MSE: 1094094.4295870159[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002052 seconds.
You can set `force_row

In [599]:
df_ = df.copy() 
mse = (modeling(preprocessing_for_train(df_, [1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1])))
print((mse))
print(math.sqrt(mse))

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003820 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 201
[LightGBM] [Info] Number of data points in the train set: 58188, number of used features: 25
[LightGBM] [Info] Start training from score 1126.812143
534907.835966243
731.3739371663738


In [600]:
df_test = pd.read_csv('test.csv')
df_test.drop(columns=['ID'], inplace=True)

In [601]:
print(df_test)

       timestamp item corporation location
0     2023-03-04   TG           A        J
1     2023-03-05   TG           A        J
2     2023-03-06   TG           A        J
3     2023-03-07   TG           A        J
4     2023-03-08   TG           A        J
...          ...  ...         ...      ...
1087  2023-03-27   RD           F        J
1088  2023-03-28   RD           F        J
1089  2023-03-29   RD           F        J
1090  2023-03-30   RD           F        J
1091  2023-03-31   RD           F        J

[1092 rows x 4 columns]


In [602]:
def put_avg_cal (df, col, range_list, avg_list) :
    count1 = 0
    for item_type in ['TG', 'CR', 'CB', 'RD', 'BC']:
        count2 = 0
        for i in range_list :
            col_name = col + '_avg'
            df.loc[(df['item'] == item_type) & (df[col] == i), col_name] = avg_list[count1][count2]
            count2 = count2 + 1
        count1 = count1 + 1

In [603]:
def preprocessing_for_test(df, whether) :
    
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['year'] = pd.to_datetime(df['timestamp']).dt.year
    df['month'] = pd.to_datetime(df['timestamp']).dt.month
    df['day'] = pd.to_datetime(df['timestamp']).dt.day
    df['day_of_week'] = df['timestamp'].dt.day_name()
    
    if whether[0] == 1 :
        put_avg_cal(df, 'month', range_list_m, avg_m)
    
    if whether[1] == 1 : 
        put_avg_cal(df, 'year', range_list_y, avg_y)
    
    if whether[2] == 1 :
        put_avg_cal(df, 'day_of_week', range_list_d, avg_d)  
        
    if whether[3] == 1 :
        put_avg_cal(df, 'location', range_list_l, avg_l) 
        
    if whether[4] == 1 :
        put_avg_cal(df, 'corporation', range_list_c, avg_c)     
        
    if whether[5] == 1 :
        df['sin_timestamp'] = np.sin(2 * np.pi * df['timestamp'].dt.dayofyear / 365)
        df['cos_timestamp'] = np.cos(2 * np.pi * df['timestamp'].dt.dayofyear / 365)
        
    if whether[6] == 0 :
        df.drop(columns=['month'], inplace=True)
        
    if whether[7] == 0 :
        df.drop(columns=['day'], inplace=True)
        
    if whether[8] == 0 :
        df.drop(columns=['year'], inplace=True) 
        
    if whether[9] == 1:
        df = pd.get_dummies(df, columns=['corporation'])
    else:
        df.drop(columns=['corporation'], inplace=True)
    
    if whether[10] == 1 :
        df = pd.get_dummies(df, columns=['location']) 
    else :  
        df.drop(columns=['location'], inplace=True)    
        
    if whether[11] == 1 :
        df = pd.get_dummies(df, columns=['day_of_week']) 
    else :  
        df.drop(columns=['day_of_week'], inplace=True)     
    
    # df.drop(columns=['timestamp'], inplace=True) 
    df = pd.get_dummies(df, columns=['item']) 
    
    return df

In [604]:
df_ = preprocessing_for_train(df_, [1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1])

col_list = df_.columns
col_list = [col for col in col_list if col != 'price(원/kg)' and col != 'timestamp']

X = df_[col_list]
Y = df_['price(원/kg)']

In [605]:
df_test = preprocessing_for_test(df_test, [1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1])

df__ = df_test.copy()
df__['timestamp'] = pd.to_datetime(df__['timestamp'])
sunday_indices = df__[(df__['timestamp'].dt.dayofweek == 6)].index.tolist()

col_list = df_test.columns
col_list = [col for col in col_list if col != 'price(원/kg)' and col != 'timestamp']

X2 = df_test[col_list]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X2)

model_lgbm = LGBMRegressor()
model_lgbm.fit(X_train_scaled, Y)

predictions_lgbm = model_lgbm.predict(X_test_scaled)
predictions_lgbm[sunday_indices] = 0
print(predictions_lgbm)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004105 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 193
[LightGBM] [Info] Number of data points in the train set: 59397, number of used features: 24
[LightGBM] [Info] Start training from score 1131.680674
[3285.95804132    0.         3498.62893591 ...  517.8313954   517.8313954
  520.40820926]


In [606]:
submission = pd.read_csv('sample_submission.csv')
submission['answer'] = predictions_lgbm
submission.to_csv('sample_submission2.csv', index=False)